# TensorFlow Setup Helper (macOS Apple Silicon · Windows 11 · Windows 11 WSL)

This notebook detects your environment and shows **safe, up-to-date commands** to create a clean environment for **TensorFlow, Keras, and NumPy**, then verifies the install.

**Supported scenarios**
- **macOS (Apple Silicon, arm64) with Conda** (uses `pip install tensorflow-macos tensorflow-metal` inside a conda env)
- **Windows 11 with Conda** (uses `pip install tensorflow` inside a conda env; CPU by default)
- **Windows 11 with WSL (Ubuntu) + pip/venv** (uses Python `venv` + `pip install tensorflow`)

> **Tip:** In VS Code, after creating an environment, press **Cmd/Ctrl+Shift+P**, then type **"Python: Select Interpreter"** and choose the new environment before running the verification cells.

In [1]:
import os, sys, platform, shutil, subprocess, json

def detect_env():
    info = {}
    info["python_exe"] = sys.executable
    info["python_version"] = sys.version.split()[0]
    info["platform_system"] = platform.system()
    info["platform_release"] = platform.release()
    info["platform_machine"] = platform.machine().lower()
    info["conda_prefix"] = os.environ.get("CONDA_PREFIX") or ""
    info["conda_default_env"] = os.environ.get("CONDA_DEFAULT_ENV") or ""
    info["is_wsl"] = False
    try:
        uname = subprocess.check_output(["uname", "-r"], text=True).lower()
        info["is_wsl"] = "microsoft" in uname or "wsl" in uname
    except Exception:
        pass
    info["is_macos"] = info["platform_system"] == "Darwin"
    info["is_apple_silicon"] = info["is_macos"] and info["platform_machine"] in ("arm64", "aarch64")
    info["is_windows"] = info["platform_system"] == "Windows"
    return info

env_info = detect_env()
print(json.dumps(env_info, indent=2))

{
  "python_exe": "C:\\Users\\prate\\AppData\\Local\\Microsoft\\WindowsApps\\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\\python.exe",
  "python_version": "3.10.11",
  "platform_system": "Windows",
  "platform_release": "10",
  "platform_machine": "amd64",
  "conda_prefix": "",
  "conda_default_env": "",
  "is_wsl": false,
  "is_macos": false,
  "is_apple_silicon": false,
  "is_windows": true
}


In [2]:
from textwrap import dedent

def recommended_commands(info):
    cmds = []
    if info["is_macos"] and info["is_apple_silicon"]:
        cmds.append(("macOS Apple Silicon + Conda",
        dedent("""
        # 1) Create a clean conda env (Python 3.11 recommended)
        conda create -n tf-mac-metal python=3.11 -y
        conda activate tf-mac-metal

        # 2) Upgrade pip tooling inside the env
        python -m pip install --upgrade pip setuptools wheel

        # 3) Install TensorFlow for macOS + Metal plugin (via pip)
        pip install tensorflow-macos
        pip install tensorflow-metal

        # 4) Install numpy (conda-forge is fine) — or just use pip
        conda install -c conda-forge numpy
        # (Alternatively) pip install numpy

        # 5) Verify inside this env by launching Jupyter/VS Code
        #    In VS Code: Python: Select Interpreter -> tf-mac-metal
        """)))
    elif info["is_windows"] and not info["is_wsl"]:
        cmds.append(("Windows 11 + Conda (CPU)",
        dedent("""
        # 1) Create a clean conda env (Python 3.11 recommended)
        conda create -n tf-win python=3.11 -y
        conda activate tf-win

        # 2) Upgrade pip tooling inside the env
        python -m pip install --upgrade pip setuptools wheel

        # 3) Install CPU TensorFlow via pip
        pip install tensorflow

        # 4) Install numpy (prefer conda-forge for speed/stability) — or pip
        conda install -c conda-forge numpy
        # (Alternatively) pip install numpy

        # 5) Verify inside this env by launching Jupyter/VS Code
        #    In VS Code: Python: Select Interpreter -> tf-win
        """)))
        cmds.append(("Windows GPU (optional guidance)",
        dedent("""
        # For GPU on Windows, the recommended path is WSL2 + NVIDIA CUDA in WSL.
        # Native Windows GPU support is limited; prefer the WSL instructions below.
        """)))
    elif info["is_wsl"]:
        cmds.append(("Windows 11 WSL (Ubuntu) + pip/venv (CPU)",
        dedent("""
        # 1) Ensure Python and venv are installed
        sudo apt update
        sudo apt install -y python3 python3-venv python3-pip

        # 2) Create and activate a venv (Python 3.11+ recommended)
        python3 -m venv ~/tf-wsl
        source ~/tf-wsl/bin/activate

        # 3) Upgrade pip tooling
        python -m pip install --upgrade pip setuptools wheel

        # 4) Install TensorFlow (CPU) and NumPy
        pip install tensorflow numpy

        # 5) Verify inside this venv by launching Jupyter/VS Code (Remote - WSL)
        #    In VS Code: Python: Select Interpreter -> ~/tf-wsl/bin/python

        # (Optional, GPU in WSL requires NVIDIA drivers + CUDA in WSL;
        #  follow official TF + NVIDIA docs if you need GPU.)
        """)))
    else:
        cmds.append(("Generic (fallback)",
        dedent("""
        # Could not confidently detect one of the targeted setups.
        # Choose the closest match or ask your instructor.
        """)))
    return cmds

cmds = recommended_commands(env_info)
for title, body in cmds:
    print("="*80)
    print(title)
    print("-"*80)
    print(body)
    print()

Windows 11 + Conda (CPU)
--------------------------------------------------------------------------------

# 1) Create a clean conda env (Python 3.11 recommended)
conda create -n tf-win python=3.11 -y
conda activate tf-win

# 2) Upgrade pip tooling inside the env
python -m pip install --upgrade pip setuptools wheel

# 3) Install CPU TensorFlow via pip
pip install tensorflow

# 4) Install numpy (prefer conda-forge for speed/stability) — or pip
conda install -c conda-forge numpy
# (Alternatively) pip install numpy

# 5) Verify inside this env by launching Jupyter/VS Code
#    In VS Code: Python: Select Interpreter -> tf-win


Windows GPU (optional guidance)
--------------------------------------------------------------------------------

# For GPU on Windows, the recommended path is WSL2 + NVIDIA CUDA in WSL.
# Native Windows GPU support is limited; prefer the WSL instructions below.




In [3]:
from pathlib import Path

candidates = [Path.cwd()]
downloads = Path.home() / "Downloads"
if downloads.exists():
    candidates.append(downloads)
mnt_data = Path("/mnt/data")
if mnt_data.exists():
    candidates.append(mnt_data)

env_macos = """
name: tf-mac-metal
channels:
  - conda-forge
dependencies:
  - python=3.11
  - pip
  - numpy
  - pip:
      - tensorflow-macos
      - tensorflow-metal
""".strip() + "\n"

env_windows = """
name: tf-win
channels:
  - conda-forge
dependencies:
  - python=3.11
  - pip
  - numpy
  - pip:
      - tensorflow
""".strip() + "\n"

req_wsl = """
tensorflow
numpy
""".strip() + "\n"

files = {
    "environment_macos_arm64.yml": env_macos,
    "environment_windows_cpu.yml": env_windows,
    "requirements_wsl.txt": req_wsl,
    "README_SETUP_STEPS.txt": (
        "This repository contains helper files for setting up TensorFlow + NumPy.\n\n"
        "macOS Apple Silicon (Conda):\n"
        "  conda env create -f environment_macos_arm64.yml\n"
        "  conda activate tf-mac-metal\n\n"
        "Windows 11 (Conda CPU):\n"
        "  conda env create -f environment_windows_cpu.yml\n"
        "  conda activate tf-win\n\n"
        "Windows 11 WSL (pip/venv):\n"
        "  python3 -m venv ~/tf-wsl\n"
        "  source ~/tf-wsl/bin/activate\n"
        "  python -m pip install --upgrade pip setuptools wheel\n"
        "  pip install -r requirements_wsl.txt\n"
    )
}

written = []
for base in candidates:
    base.mkdir(parents=True, exist_ok=True)
    for fname, content in files.items():
        p = base / fname
        p.write_text(content, encoding="utf-8")
        written.append(str(p))

print("Wrote the following files:")
for p in written:
    print(" -", p)

Wrote the following files:
 - c:\Users\prate\Documents\Perdue ML classes\PC-AIML\Deep Learning\class7\environment_macos_arm64.yml
 - c:\Users\prate\Documents\Perdue ML classes\PC-AIML\Deep Learning\class7\environment_windows_cpu.yml
 - c:\Users\prate\Documents\Perdue ML classes\PC-AIML\Deep Learning\class7\requirements_wsl.txt
 - c:\Users\prate\Documents\Perdue ML classes\PC-AIML\Deep Learning\class7\README_SETUP_STEPS.txt
 - C:\Users\prate\Downloads\environment_macos_arm64.yml
 - C:\Users\prate\Downloads\environment_windows_cpu.yml
 - C:\Users\prate\Downloads\requirements_wsl.txt
 - C:\Users\prate\Downloads\README_SETUP_STEPS.txt


## Verify your install

Run the cell below **after** you've selected the newly created environment in VS Code.

In [4]:
import importlib, traceback, sys, os

def try_import(modname):
    try:
        m = importlib.import_module(modname)
        return True, m, None
    except Exception as e:
        return False, None, e

results = {}

ok_np, np, e_np = try_import("numpy")
results["numpy"] = (ok_np, str(e_np) if e_np else "")

ok_tf, tf, e_tf = try_import("tensorflow")
results["tensorflow"] = (ok_tf, str(e_tf) if e_tf else "")

ok_keras, keras, e_keras = try_import("keras")
results["keras (optional)"] = (ok_keras, str(e_keras) if e_keras else "")

print("Python:", sys.version.split()[0])
print("Interpreter:", sys.executable)
print()

for name, (ok, err) in results.items():
    print(f"{name:18} -> {'OK' if ok else 'NOT FOUND / ERROR'}")
    if not ok and err:
        print("  details:", err[:400], "..." if len(err)>400 else "")
print()

if ok_tf:
    print("TensorFlow version:", tf.__version__)
    try:
        devs = tf.config.list_physical_devices()
        print("Physical devices:", devs)
        print("GPUs:", tf.config.list_physical_devices('GPU'))
    except Exception as e:
        print("Device query error:", e)
    try:
        import numpy as _np
        a = tf.constant(_np.random.randn(256, 256), dtype=tf.float32)
        b = tf.constant(_np.random.randn(256, 256), dtype=tf.float32)
        c = tf.matmul(a, b)
        print("Matmul OK. Tensor shape:", c.shape, "dtype:", c.dtype)
    except Exception as e:
        print("Matmul failed:", e)

if ok_keras:
    print("Keras version:", keras.__version__)
    # If using Keras 3 with TensorFlow backend, you can ensure:
    # os.environ['KERAS_BACKEND'] = 'tensorflow'  # set before importing keras

Python: 3.10.11
Interpreter: C:\Users\prate\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe

numpy              -> OK
tensorflow         -> OK
keras (optional)   -> OK

TensorFlow version: 2.17.0
Physical devices: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]
GPUs: []
Matmul OK. Tensor shape: (256, 256) dtype: <dtype: 'float32'>
Keras version: 3.2.0


### (Optional) Mini-training sanity check

This builds a tiny model and trains for one epoch on random data to confirm that TF + (tf.)keras works.
Skip if you're low on time or memory.

In [5]:
try:
    import numpy as np
    import tensorflow as tf
    from tensorflow import keras
    from tensorflow.keras import layers

    X = np.random.randn(1024, 10).astype("float32")
    y = (np.random.rand(1024) > 0.5).astype("int32")

    model = keras.Sequential([
        layers.Input(shape=(10,)),
        layers.Dense(32, activation="relu"),
        layers.Dense(1, activation="sigmoid"),
    ])
    model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
    hist = model.fit(X, y, epochs=1, batch_size=32, verbose=0)
    print("Training OK. Final metrics:", {k: float(v[-1]) for k, v in hist.history.items()})
except Exception as e:
    print("Mini-training failed:", e)
    print("Tip: ensure you are in the environment you created and that TensorFlow is installed there.")

Training OK. Final metrics: {'accuracy': 0.5087890625, 'loss': 0.7337875366210938}


## Troubleshooting (quick fixes)

**macOS (Apple Silicon): `NotFoundError: ... libmetal_plugin.dylib`**  
- Cause: Mixed `conda` and `pip` TF installs, or unsupported Python version.  
- Fix: Use a *fresh* env with `python=3.11` → `pip install tensorflow-macos tensorflow-metal` (do **not** `conda install tensorflow*`).

**Windows 11 (Conda): install succeeds but import fails**  
- Ensure you installed with **pip** inside the conda env: `pip install tensorflow` (CPU). Avoid `conda install tensorflow`.  
- Verify VS Code is using the env: **Python: Select Interpreter** → pick your env.

**WSL: GPU not detected**  
- That's expected on CPU-only installs. For GPU, you need recent NVIDIA drivers on Windows + CUDA inside WSL + TF GPU build. Start CPU-first.

**General**  
- Avoid mixing package managers: prefer **conda for the env + numpy**, and **pip for TensorFlow**.  
- Use **one env per project**. If things get weird, recreate the env.  
- If you want standalone `keras` (Keras 3), install with `pip install keras` and set `KERAS_BACKEND=tensorflow` *before* importing `keras`.